# **CATBOOST**

* CatBoost는 Categorical Boosting으로 범주형 feature를 처리하는데 중점을 둔 알고리즘이다.
* Yandex에 의해 개발되었고, Gradient Boosting에 기반한다.
* 기존에 존재하던 boosting 모델인 XGBoost, Light GBM을 능가하는 머신러닝 기법이다.

#### 범주형 변수의 타깃 인코딩
* 범주형 변수로서 지정한 특징을 자동으로 타깃 인코딩하여 수치로 변환한다.
* 어떤 분기에서 범주형 변수가 사용되었을 때 그 범주형 변수와 다른 범주형 변수와의 조합에 대해 타깃 인코딩 연산이 이루어지며 그보다 깊은 분기에서 그 결과가 사용된다.

#### 망각 결정 트리
* 각 깊이별 분기 조건식이 모두 망각 결정 트리 oblivious decision tree라 불리는 결정 트리를 사용한다.

#### 정렬 부스팅
* 데이터 수가 적을 때는 정렬 부스팅 ordered boosting이라는 알고리즘이 사용된다. 속도는 느리지만 데이터 수가 적을 때는 모델 성능이 높다고 한다.

#### **장점**
* 기존의 GBM기반 알고리즘은 이전 데이터를 반복적으로 쓰는 과정에서 타깃 누수(target leakage)와 예측 결과 변화 문제를 일으키고 범주형 변수를 수치형 변수로 전처리 해준 후 일일이 훈련해줘야 한다는 문제가 있다. CatBoost는 Ordered boosting과 새로운 범주형 변수 처리 방법으로 이를 해결한다.
* 기존의 XGBoost와 같은 부스팅은 오래걸리며 하이퍼 파라미터 튜닝에 매우 민감하다. CatBoost는 파라미터 튜닝에 큰 신경을 쓰지 않아도 되고, 학습 속도도 상대적으로 빠르다.

#### **한계점**
* CatBoost는 밀도가 넓고 드문 Sparse한 matrix는 처리하지 못 한다.
* 데이터 대부분이 수치형인 경우 Light GBM보다 학습 속도가 느리다.
* 때문에 가급적 범주형 변수인 경우 사용하는 것이 좋다.

[출처]
* 데이터가 뛰어노는 AI 놀이터, 캐글
* https://blog.naver.com/remedies/222455892119

### 01. 라이브러리 불러오기

In [1]:
import numpy as np
import pandas as pd
import os
import gc

### 02. 데이터 불러오기

In [2]:
train = pd.read_pickle("../input/ae-credit-id-encoded-dataset-fp16/id_encoded_fp16_train_data.pkl")
label = pd.read_pickle("../input/ae-credit-id-encoded-dataset-fp16/id_encoded_train_labels.pkl")

#### pickle 모듈
* 객체를 직렬화하여 파일에 저장하여 파이썬 객체 구조를 직렬화 및 역직렬화 하는데 사용한다.
* pickle 모듈로 저장된 파일은 binary 형태로 저장되어 속도가 빠르다.

In [3]:
# https://www.kaggle.com/competitions/amex-default-prediction/discussion/327094
train =  (train
            .groupby('customer_ID')
            .tail(1)
            .set_index('customer_ID', drop=True)
            .sort_index()
            .drop(['S_2'], axis='columns'))

In [4]:
train = pd.merge(train, label, how="left", on="customer_ID")

In [5]:
train.shape

In [6]:
all_cols = train.columns.to_list()
all_cols

In [7]:
cat_cols = ['B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_63', 'D_64', 'D_66', 'D_68']

In [8]:
num_cols = [col for col in all_cols if col not in cat_cols + ["target"]]

In [9]:
num_cols

In [10]:
train_X = train[cat_cols + num_cols]
train_y = pd.DataFrame(train["target"])

In [11]:
for col in cat_cols:
    train_X[col] = train_X[col].astype(str)

### 03. 경쟁 지표 찍어주기

In [12]:
def amex_metric(y_true: pd.DataFrame, y_pred: pd.DataFrame) -> float:

    def top_four_percent_captured(y_true: pd.DataFrame, y_pred: pd.DataFrame) -> float:
        df = (pd.concat([y_true, y_pred], axis='columns')
              .sort_values('prediction', ascending=False))
        df['weight'] = df['target'].apply(lambda x: 20 if x==0 else 1)
        four_pct_cutoff = int(0.04 * df['weight'].sum())
        df['weight_cumsum'] = df['weight'].cumsum()
        df_cutoff = df.loc[df['weight_cumsum'] <= four_pct_cutoff]
        return (df_cutoff['target'] == 1).sum() / (df['target'] == 1).sum()
        
    def weighted_gini(y_true: pd.DataFrame, y_pred: pd.DataFrame) -> float:
        df = (pd.concat([y_true, y_pred], axis='columns')
              .sort_values('prediction', ascending=False))
        df['weight'] = df['target'].apply(lambda x: 20 if x==0 else 1)
        df['random'] = (df['weight'] / df['weight'].sum()).cumsum()
        total_pos = (df['target'] * df['weight']).sum()
        df['cum_pos_found'] = (df['target'] * df['weight']).cumsum()
        df['lorentz'] = df['cum_pos_found'] / total_pos
        df['gini'] = (df['lorentz'] - df['random']) * df['weight']
        return df['gini'].sum()

    def normalized_weighted_gini(y_true: pd.DataFrame, y_pred: pd.DataFrame) -> float:
        y_true_pred = y_true.rename(columns={'target': 'prediction'})
        return weighted_gini(y_true, y_pred) / weighted_gini(y_true, y_true_pred)

    g = normalized_weighted_gini(y_true, y_pred)
    d = top_four_percent_captured(y_true, y_pred)

    return 0.5 * (g + d)

### 04. Competition metric을 Testing 하기

In [13]:
y_pred = train_y.copy(deep=True).rename(columns={'P_2': 'prediction'}).drop("target",axis=1)
y_pred["prediction"] = 0
y_pred.head()

In [14]:
amex_metric(train_y,y_pred)

### 05. CatBoostClassifier 사용하여 기본 제출 만들기

In [15]:
import lightgbm as lgb
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test = train_test_split(train_X,train_y,stratify=train_y)
clf = CatBoostClassifier(iterations=1000)

clf.fit(X_train,y_train,eval_set=[(X_test,y_test)],cat_features=cat_cols,verbose=100)

In [16]:
y_pred = y_test.copy(deep=True)
y_pred = y_pred.rename(columns={"target":"prediction"})
y_pred["prediction"] = clf.predict_proba(X_test)[:,1]

In [17]:
amex_metric(y_test,y_pred) # Metric calculation on validation set

### 06. Test data 예측하기

In [23]:
del train, train_X, train_y, X_train, X_test, y_train, y_test
gc.collect()

In [24]:
test = pd.read_pickle("../input/ae-credit-id-encoded-dataset-fp16/id_encoded_fp16_test_data.pkl")

In [25]:
test =  (test
         .groupby('customer_ID')
         .tail(1)
         .set_index('customer_ID', drop=True)
         .sort_index()
         .drop(['S_2'], axis='columns'))

In [26]:
for col in cat_cols:
    test[col] = test[col].astype(str)

In [27]:
test["prediction"] = clf.predict_proba(test[cat_cols + num_cols])[:,1]

In [28]:
test.head()

In [29]:
from sklearn.preprocessing import LabelEncoder
loaded_encoder = LabelEncoder()
loaded_encoder.classes_ = np.load(f"../input/ae-credit-id-encoded-dataset-fp16/id_encodings.npy", allow_pickle=True)

In [30]:
test = test.reset_index()

In [31]:
test["customer_ID"] = loaded_encoder.inverse_transform(test["customer_ID"])
test.head()

In [32]:
test[["customer_ID", "prediction"]].to_csv("submission_first.csv",index=False) #Creating submission file